In [17]:
import yaml
import os

def parse_yaml(file_path, encoding='utf-8'):
	assert os.path.isfile(file_path)
	with open(file_path, "r", encoding=encoding) as f:
		ret = yaml.load(f, Loader=yaml.FullLoader)
	return ret

# main database
db_list = parse_yaml("./ps_db.yaml")
# site link dictionary
st_dict = { info['site_name']: info['link_prefix'] for info in parse_yaml("./ps_md.yaml") }

print(db_list)
print(st_dict)

[{'title': '두 배열의 합', 'id': 2143, 'review_note': None, 'site': '백준', 'rank': 'G3', 'tags': ['two-pointer', 'implementation'], 'records': [{'date': datetime.date(2023, 12, 21), 'state': False, 'time': 60}]}, {'title': '요세푸스 문제 0', 'id': 11866, 'review_note': None, 'site': '백준', 'rank': 'S5', 'tags': ['circular-queue'], 'records': [{'date': datetime.date(2024, 1, 11), 'state': True, 'time': 45}]}, {'title': '좌표 정렬하기', 'id': 11650, 'review_note': None, 'site': '백준', 'rank': 'S5', 'tags': ['stable-sort'], 'records': [{'date': datetime.date(2024, 1, 11), 'state': True, 'time': 9}]}, {'title': '덱', 'id': 10866, 'review_note': None, 'site': '백준', 'rank': 'S4', 'tags': ['deque'], 'records': [{'date': datetime.date(2024, 1, 11), 'state': True, 'time': 90}]}, {'title': '특정한 최단 경로', 'id': 1504, 'site': '백준', 'rank': 'G4', 'tags': ['shortest-path', 'dijkstra'], 'review_note': True, 'records': [{'date': datetime.date(2024, 1, 18), 'state': True, 'time': 72, 'note': '두 정점 방문 순서가 상관 없다는걸 나중에 읽음'}]}, 

In [18]:
import pandas as pd

db_df = pd.DataFrame(db_list)
# ['site', 'id', 'title', 'rank']

db_display = pd.DataFrame() # Empty Dataframe


In [19]:
# make unique title

def make_unique_title(df_row):
	title = df_row['title']
	rank = df_row['rank']
	site_name = df_row['site']
	id = str(df_row['id'])

	if (site_name == "LeetCode"):
		str_id = '-'.join((title.lower()).split())
		link = st_dict.get(site_name) + str_id + "/description/"
	else:
		link = st_dict.get(site_name) + id
	return f'<a href={link}>{site_name} {id} {title} {rank}</a>'

db_display['unique_name'] = db_df.apply(make_unique_title, axis=1)



In [20]:
# make tag list

def make_tag_list(df_row):
	tags = df_row['tags']
	if (isinstance(tags, list)):
		tag_list = ''.join([f'<li>{tag}</li>' for tag in tags])
	else:
		tag_list = f'<li>{tags}</li>'
	return f'<ul>{tag_list}</ul>'

db_display['tag_list'] = db_df.apply(make_tag_list, axis=1)


In [21]:
# make rank

def make_rank(df_row):
	rank = df_row['rank']
	return rank

db_display['rank'] = db_df.apply(make_rank, axis=1)


In [22]:
# all records

scc = 'green'
sfc = 'red'

def make_all_records(df_row):
	records = df_row['records']
	records_str = str()
	for record in records:
		date = record['date']
		state = 'S' if record['state'] else 'F'
		state = f'''<p style="color: {scc if record['state'] else sfc}">{state}</p>'''
		time = str(record['time'])
		records_str += f'<tr><td>{date}</td><td>{state}</td><td>{time:>5}</td></tr>'
	records_str = f'<tr>{records_str}</tr>'
	table_header = '<tr><th>날짜</th><th>결과</th><th>시간</th></tr>'
	return f'<table>{table_header}{records_str}</table>'

db_display['all_records'] = db_df.apply(make_all_records, axis=1)


In [23]:
# make score

def make_score(df_row):
	records = df_row['records']
	total_count = len(records)
	success_count = sum(1 for record in records if record['state'])
	percentage = success_count / total_count * 100
	return f'{percentage:.0f}%'

db_display['score'] = db_df.apply(make_score, axis=1)


In [24]:
# make dates before
from datetime import datetime

def make_dates_before(df_row):
	records = df_row['records']
	today = datetime.today().date()
	latest = max([record['date'] for record in records])
	return today - latest

db_display['dates_before'] = db_df.apply(make_dates_before, axis=1)

In [25]:
# make review_note

def make_review_note(df_row):
	records = df_row['review_note']
	if records == True:
		color = "palegreen"
		ret = 'Done'
	elif records == False:
		color = "pink"
		ret = 'Required'
	else:
		color = "dimgrey"
		ret = 'Not Required'
	return f'<p style="color: {color}">{ret}</p>'

db_display['review_note'] = db_df.apply(make_review_note, axis=1)

In [26]:
from IPython.display import display, HTML

# sort
db_display = db_display.sort_values(by='dates_before', ascending=True)

# style
db_style = 'style="font-size: 13px"'
db_display = db_display.style.set_table_attributes(db_style)

#display
display(HTML(db_display.to_html(escape=False)))